# Learning Fair Representations (LFR)

#### Setting up enviornment in Colab

In [20]:
!pip3 install aif360

/bin/sh: pip3: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

#### This notebook demonstrates the use of the learning fair representations algorithm for bias mitigation
Learning fair representations [1] is a pre-processing technique that finds a latent representation which encodes the data well but obfuscates information about protected attributes. We will see how to use this algorithm for learning representations that encourage individual fairness and apply them on the Adult dataset.

In [21]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

#### Load dataset and set options

In [22]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

#### Clean up training data

In [23]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [24]:
# Metric for the original dataset
privileged_groups = [{'sex': 1.0}]
unprivileged_groups = [{'sex': 0.0}]
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.193332


#### Train with and transform the original training data

In [25]:
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
    
TR = LFR(unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
TR = TR.fit(dataset_orig_train)

10250 25155.290358962993
10500 20000.175153017888
10750 19490.27764287732
11000 20860.54150699525
11250 18672.976576776302
11500 17901.292574721065
11750 17468.024144880725
12000 17156.65782257696
12250 16478.17646545546
12500 16233.736621917198
12750 16018.27147107636
13000 15935.443107312132
13250 15772.052204292
13500 15746.076778099863
13750 15735.228710715623
14000 15705.685081010024
14250 15651.450660702572
14500 15608.71714045799
14750 15546.45241402367
15000 15466.17698518403


In [8]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_train)

#### Metric with the transformed training data

In [9]:
from sklearn.metrics import classification_report
thresholds = [0.2, 0.3, 0.35, 0.4, 0.5]
for threshold in thresholds:
    
    # Transform training data and align features
    dataset_transf_train = TR.transform(dataset_orig_train, threshold=threshold)

    metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    display(Markdown("#### Transformed training dataset"))
    print("Classification threshold = %f" % threshold)
    #print(classification_report(dataset_orig_train.labels, dataset_transf_train.labels))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Classification threshold = 0.200000
Difference in mean outcomes between unprivileged and privileged groups = -0.495945


#### Transformed training dataset

Classification threshold = 0.300000
Difference in mean outcomes between unprivileged and privileged groups = -0.345693


#### Transformed training dataset

Classification threshold = 0.350000
Difference in mean outcomes between unprivileged and privileged groups = -0.340769


#### Transformed training dataset

Classification threshold = 0.400000
Difference in mean outcomes between unprivileged and privileged groups = -0.261575


#### Transformed training dataset

Classification threshold = 0.500000
Difference in mean outcomes between unprivileged and privileged groups = -0.214970


Optimized preprocessing has reduced the disparity in favorable outcomes between the privileged and unprivileged
groups (training data).

In [26]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_transf_train.consistency())
print("Consistency of labels in original training dataset= %f" %metric_orig_train.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 1.000000
Consistency of labels in original training dataset= 0.746509


In [11]:
## PCA Analysis of consistency

In [27]:
import pandas as pd

feat_cols = dataset_orig_train.feature_names

orig_df = pd.DataFrame(dataset_orig_train.features,columns=feat_cols)
orig_df['label'] = dataset_orig_train.labels
orig_df['label'] = orig_df['label'].apply(lambda i: str(i))

transf_df = pd.DataFrame(dataset_transf_train.features,columns=feat_cols)
transf_df['label'] = dataset_transf_train.labels
transf_df['label'] = transf_df['label'].apply(lambda i: str(i))

In [28]:
from sklearn.decomposition import PCA

orig_pca = PCA(n_components=3)
orig_pca_result = orig_pca.fit_transform(orig_df[feat_cols].values)

orig_df['pca-one'] = orig_pca_result[:,0]
orig_df['pca-two'] = orig_pca_result[:,1] 
orig_df['pca-three'] = orig_pca_result[:,2]

display(Markdown("#### Original training dataset"))
print('Explained variation per principal component:')
print(orig_pca.explained_variance_ratio_)

#### Original training dataset

Explained variation per principal component:
[0.15361504 0.1471127  0.125872  ]


In [29]:
transf_pca = PCA(n_components=3)
transf_pca_result = transf_pca.fit_transform(transf_df[feat_cols].values)

transf_df['pca-one'] = transf_pca_result[:,0]
transf_df['pca-two'] = transf_pca_result[:,1] 
transf_df['pca-three'] = transf_pca_result[:,2]

display(Markdown("#### Transformed training dataset"))
print('Explained variation per principal component:')
print(transf_pca.explained_variance_ratio_)

#### Transformed training dataset

Explained variation per principal component:
[0.62040504 0.37140779 0.00810437]


#### Load, clean up original test data and compute metric

In [30]:
display(Markdown("#### Testing Dataset shape"))
print(dataset_orig_test.features.shape)

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Testing Dataset shape

(14653, 18)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.197229


#### Transform test data and compute metric

In [32]:
dataset_transf_test = TR.transform(dataset_orig_test, threshold=threshold)
metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

In [33]:
print("Consistency of labels in tranformed test dataset= %f" %metric_transf_test.consistency())

Consistency of labels in tranformed test dataset= 1.000000


In [34]:
print("Consistency of labels in original test dataset= %f" %metric_orig_test.consistency())

Consistency of labels in original test dataset= 0.730936


In [35]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_transf_test.consistency() > metric_orig_test.consistency(), "Transformed dataset consistency should be greater than original dataset."

check_algorithm_success()    

    References:
    [1] R. Zemel, Y. Wu, K. Swersky, T. Pitassi, and C. Dwork,  "Learning Fair Representations." 
    International Conference on Machine Learning, 2013.